In [1]:
# Create the directory if it doesn't exist
!mkdir -p ../data

# Download files using curl
!curl -o ../data/green_tripdata_2024-01.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet
!curl -o ../data/green_tripdata_2024-02.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-02.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1330k  100 1330k    0     0  1880k      0 --:--:-- --:--:-- --:--:-- 1879k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1253k  100 1253k    0     0  1143k      0  0:00:01  0:00:01 --:--:-- 1143k


In [2]:
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer
from sklearn.linear_model import Lasso, Ridge, LinearRegression

In [3]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [4]:
df_train = read_dataframe('../data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2024-02.parquet')

In [5]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [6]:
categorical = ['PU_DO']  #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [7]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [8]:
import dagshub
import mlflow


dagshub.init(url="https://dagshub.com/daduke1/nyc-taxi-time-prediction", mlflow=True)

MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()

print(MLFLOW_TRACKING_URI)

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="nyc-taxi-experiment")

Accessing as daduke1

Initialized MLflow to track repo "daduke1/nyc-taxi-time-prediction"

Repository daduke1/nyc-taxi-time-prediction initialized!

https://dagshub.com/daduke1/nyc-taxi-time-prediction.mlflow


2024/09/20 17:38:33 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/655077ef21ea4d658c388a4f45673d3f', creation_time=1726875513781, experiment_id='0', last_update_time=1726875513781, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [9]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2024-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2024-02")

In [10]:
from dagshub import get_repo_bucket_client
# Get a boto3.client object
s3 = get_repo_bucket_client("daduke1/nyc-taxi-time-prediction")

# Upload train file
s3.upload_file(
    Bucket="nyc-taxi-time-prediction",  # name of the repo
    Filename="../data/green_tripdata_2024-01.parquet",  # local path of file to upload
    Key="train_data.parquet",  # remote path where to upload the file
)

# Upload test file
s3.upload_file(
    Bucket="nyc-taxi-time-prediction",  # name of the repo
    Filename="../data/green_tripdata_2024-02.parquet",  # local path of file to upload
    Key="test_data.parquet",  # remote path where to upload the file
)

Client created. Use the name of the repo (nyc-taxi-time-prediction) as the name of the bucket

In [11]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import pathlib

In [12]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [13]:
def objective(params):
    with mlflow.start_run(nested=True):
         
        # Tag model
        mlflow.set_tag("model_family", "xgboost")
        
        # Log parameters
        mlflow.log_params(params)
        
        # Train model
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=10
        )
        
        # Log xgboost model with artifact_path
        mlflow.xgboost.log_model(booster, artifact_path="model")
         
        # Predict in the val dataset
        y_pred = booster.predict(valid)
        
        # Calculate metric
        rmse = root_mean_squared_error(y_val, y_pred)
        
        # Log performance metric
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [14]:
mlflow.xgboost.autolog()

with mlflow.start_run(run_name="Xgboost Hyper-parameter Optimization", nested=True):
    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
        'learning_rate': hp.loguniform('learning_rate', -3, 0),
        'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
        'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
        'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
        'objective': 'reg:squarederror',
        'seed': 42
    }
    
    best_params = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
    )
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["seed"] = 42
    best_params["objective"] = "reg:squarederror"
    
    mlflow.log_params(best_params)

    # Log tags
    mlflow.set_tags(
        tags={
            "project": "NYC Taxi Time Prediction Project",
            "optimizer_engine": "hyper-opt",
            "model_family": "xgboost",
            "feature_set_version": 1,
        }
    )

    # Log a fit model instance
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, 'validation')],
        early_stopping_rounds=10
    )
        
    y_pred = booster.predict(valid)
    
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    
    pathlib.Path("models").mkdir(exist_ok=True)
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

[0]	validation-rmse:8.68319                           
[1]	validation-rmse:8.29981                           
[2]	validation-rmse:7.95748                           
[3]	validation-rmse:7.65254                           
[4]	validation-rmse:7.38187                           
[5]	validation-rmse:7.14241                           
[6]	validation-rmse:6.93069                           
[7]	validation-rmse:6.74413                           
[8]	validation-rmse:6.58000                           
[9]	validation-rmse:6.43643                           
[10]	validation-rmse:6.31057                          
[11]	validation-rmse:6.20061                          
[12]	validation-rmse:6.10492                          
[13]	validation-rmse:6.02175                          
[14]	validation-rmse:5.94927                          
[15]	validation-rmse:5.88604                          
[16]	validation-rmse:5.83137                          
[17]	validation-rmse:5.78374                          
[18]	valid

2024/09/20 17:39:10 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/20 17:39:10 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/erick/Library/CloudStorage/OneDrive-ITESO/Proyecto_Ciencia_datos/nyc-taxi-time-prediction/.venv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [17:39:10] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/20 17:39:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

/Users/erick/Library/CloudStorage/OneDrive-ITESO/Proyecto_Ciencia_datos/nyc-taxi-time-prediction/.venv/lib/python3.11/s

[0]	validation-rmse:8.19907                                                    
[1]	validation-rmse:7.48301                                                    
[2]	validation-rmse:6.92805                                                    
[3]	validation-rmse:6.50596                                                    
[4]	validation-rmse:6.18497                                                    
[5]	validation-rmse:5.94424                                                    
[6]	validation-rmse:5.76598                                                    
[7]	validation-rmse:5.63403                                                    
[8]	validation-rmse:5.53618                                                    
[9]	validation-rmse:5.46528                                                    
[10]	validation-rmse:5.41253                                                   
[11]	validation-rmse:5.37408                                                   
[12]	validation-rmse:5.34391            

2024/09/20 17:39:30 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/20 17:39:30 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/erick/Library/CloudStorage/OneDrive-ITESO/Proyecto_Ciencia_datos/nyc-taxi-time-prediction/.venv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [17:39:30] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/20 17:39:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

/Users/erick/Library/CloudStorage/OneDrive-ITESO/Proyecto_Ciencia_datos/nyc-taxi-time-prediction/.venv/lib/python3.11/s

[0]	validation-rmse:8.48144                                                    
[1]	validation-rmse:7.94497                                                   
[2]	validation-rmse:7.49068                                                   
[3]	validation-rmse:7.10862                                                   
[4]	validation-rmse:6.78979                                                   
[5]	validation-rmse:6.52432                                                   
[6]	validation-rmse:6.30375                                                   
[7]	validation-rmse:6.12218                                                   
[8]	validation-rmse:5.97356                                                   
[9]	validation-rmse:5.85207                                                   
[10]	validation-rmse:5.75204                                                  
[11]	validation-rmse:5.67135                                                  
[12]	validation-rmse:5.60526                       

...

2024/09/20 17:41:51 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/20 17:41:51 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/erick/Library/CloudStorage/OneDrive-ITESO/Proyecto_Ciencia_datos/nyc-taxi-time-prediction/.venv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [17:41:51] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/20 17:41:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

/Users/erick/Library/CloudStorage/OneDrive-ITESO/Proyecto_Ciencia_datos/nyc-taxi-time-prediction/.venv/lib/python3.11/s

[0]	validation-rmse:8.63691                                                   
[1]	validation-rmse:8.21607                                                   
[2]	validation-rmse:7.84397                                                   
[3]	validation-rmse:7.51626                                                   
[4]	validation-rmse:7.22834                                                   
[5]	validation-rmse:6.97637                                                   
[6]	validation-rmse:6.75645                                                   
[7]	validation-rmse:6.56540                                                   
[8]	validation-rmse:6.39930                                                   
[9]	validation-rmse:6.25539                                                   
[10]	validation-rmse:6.13160                                                  
[11]	validation-rmse:6.02449                                                  
[12]	validation-rmse:5.93265                        

2024/09/20 17:42:11 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.

2024/09/20 17:42:11 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/erick/Library/CloudStorage/OneDrive-ITESO/Proyecto_Ciencia_datos/nyc-taxi-time-prediction/.venv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [17:42:11] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

2024/09/20 17:42:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

/Users/erick/Library/CloudStorage/OneDrive-ITESO/Proyecto_Ciencia_datos/nyc-taxi-time-prediction/.venv/lib/python3.11/s

100%|██████████| 10/10 [03:20<00:00, 20.02s/trial, best loss: 5.18397626097979]
[0]	validation-rmse:8.19907
[1]	validation-rmse:7.48301
[2]	validation-rmse:6.92805
[3]	validation-rmse:6.50596
[4]	validation-rmse:6.18497
[5]	validation-rmse:5.94424
[6]	validation-rmse:5.76598
[7]	validation-rmse:5.63403
[8]	validation-rmse:5.53618
[9]	validation-rmse:5.46528
[10]	validation-rmse:5.41253
[11]	validation-rmse:5.37408
[12]	validation-rmse:5.34391
[13]	validation-rmse:5.32260
[14]	validation-rmse:5.30632
[15]	validation-rmse:5.29399
[16]	validation-rmse:5.28505
[17]	validation-rmse:5.27685
[18]	validation-rmse:5.26992
[19]	validation-rmse:5.26387
[20]	validation-rmse:5.25911
[21]	validation-rmse:5.25493
[22]	validation-rmse:5.25166
[23]	validation-rmse:5.24833
[24]	validation-rmse:5.24587
[25]	validation-rmse:5.24338
[26]	validation-rmse:5.24001
[27]	validation-rmse:5.23809
[28]	validation-rmse:5.23631
[29]	validation-rmse:5.23559
[30]	validation-rmse:5.23398
[31]	validation-rmse:5.23335
[3

2024/09/20 17:42:30 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/09/20 17:42:30 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/erick/Library/CloudStorage/OneDrive-ITESO/Proyecto_Ciencia_datos/nyc-taxi-time-prediction/.venv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [17:42:30] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2024/09/20 17:42:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/09/20 17:42:35 INFO mlflow.tracking._tracking_service.client: 🏃 View run Xgboost Hyper-parameter Optimization at: htt

In [15]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="nyc-taxi-model"
)

Ingrese el run_id 58bc50029dfd47c7ae4043df7a1e1dc9


Successfully registered model 'nyc-taxi-model'.
2024/09/20 17:43:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-model, version 1
Created version '1' of model 'nyc-taxi-model'.


In [16]:
from datetime import datetime
from mlflow import MlflowClient

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.update_registered_model(
    name="nyc-taxi-model",
    description="Model registry for the NYC Taxi Time Prediction Project",
)

new_alias = "champion"
date = datetime.today()
model_version = "1"

# create "champion" alias for version 1 of model "nyc-taxi-model"
client.set_registered_model_alias(
    name="nyc-taxi-model",
    alias=new_alias,
    version=model_version
)

client.update_model_version(
    name="nyc-taxi-model",
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}",
)

<ModelVersion: aliases=['champion'], creation_timestamp=1726875831933, current_stage='None', description='The model version 1 was transitioned to champion on 2024-09-20 17:43:52.129992', last_updated_timestamp=1726875832448, name='nyc-taxi-model', run_id='58bc50029dfd47c7ae4043df7a1e1dc9', run_link='', source='mlflow-artifacts:/655077ef21ea4d658c388a4f45673d3f/58bc50029dfd47c7ae4043df7a1e1dc9/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [17]:
import mlflow.pyfunc

model_name = "nyc-taxi-model"
alias = "champion"

model_uri = f"models:/{model_name}@{alias}"

champion_version = mlflow.pyfunc.load_model(
    model_uri=model_uri
)

champion_version.predict(X_val)

array([19.173998 , 28.519112 ,  7.0510125, ..., 32.692772 , 13.44012  ,
       19.704332 ], dtype=float32)